In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
image_dir = Path('/content/drive/MyDrive/output/train')

In [ ]:
filepaths = pd.Series(list(image_dir.glob(r'**/*.jpg')), name='Filepath').astype(str)

k_values = pd.Series(filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='K').astype(np.int)

images = pd.concat([filepaths, k_values], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [ ]:
train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='K',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='K',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='K',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

In [ ]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
predicted_k_value = np.squeeze(model.predict(test_images))
true_k_value = test_images.labels

print(predicted_k_value)
print(true_k_value)

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_k_value, predicted_k_value)
print("Test R^2 Score: {:.5f}".format(r2))

In [ ]:
# prediciton for a single image input
import joblib
from PIL import Image
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

image_for_testing = Path('/content/drive/MyDrive/output/test/test.jpg')


prediction_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
predict_img = image.load_img(image_for_testing, target_size=(120, 120))
predict_img_array = image.img_to_array(predict_img)
predict_img_array = np.expand_dims(predict_img_array, axis=0)
predict_img_array = predict_img_array / 255.0  # Apply rescaling

predictions = model.predict(predict_img_array)

print("Predictions:", predictions)

In [ ]:
# export model as binaary

joblib.dump(model, 'k-finding-model.joblib')